# Processament de Dades per a la Classificació de Fruites Fresques i Podrides

Aquest document descriu el procés de càrrega, preprocesament i preparació de dades per a la classificació d'imatges de fruites en dues categories: **fresques** i **podrides**. L'objectiu és desenvolupar un sistema automàtic que pugui identificar l'estat de les fruites mitjançant tècniques d'extracció de característiques i classificació supervisada.

**1. Importació de Llibreries Necessàries**

S'utilitzen diverses llibreries per gestionar la lectura d'imatges, manipulació de dades, extracció de característiques i preparació per a l'entrenament del model de classificació.

**2. Organització de les Dades**

Les imatges de les fruites estan organitzades en carpetes diferenciades segons el seu estat:
- **Fruits Fresques:** Inclou subcarpetes com ara `freshapples`, `freshbanana` i `freshoranges`.
- **Fruits Podrides:** Inclou subcarpetes com ara `rottenapples`, `rottenbanana` i `rottenoranges`.

**3. Extracció de Característiques amb Histogram of Oriented Gradients (HOG)**

Per a cada imatge, el procés inclou:
- **Redimensionament:** Ajustament de la mida de la imatge a `(64, 32)` píxels per uniformitzar les dades d'entrada.
- **Conversió a Gris:** Simplificació de la imatge per reduir la complexitat en la detecció de característiques.
- **Extracció de Característiques HOG:** Captura de les gradients d'orientació per obtenir una representació robusta de les formes i textures de les fruites.

**4. Preparació dels Conjunts d'Entrenament i Prova**

Les característiques extraites es distribueixen en dos conjunts:
- **Conjunt d'Entrenament (`train`):** Utilitzat per entrenar el model de classificació.
- **Conjunt de Prova (`test`):** Utilitzat per avaluar el rendiment del model entrenat.

**5. Codificació d'Etiquetes**

Les etiquetes textuals (`"fresh"`, `"rotten"`) s'han transformat a valors numèrics mitjançant `LabelEncoder` per facilitar l'entrament dels models de classificació.

**Conclusió**

Aquest procés estableix una base sòlida per a la classificació binària de fruites basant-se en imatges, utilitzant tècniques eficaços d'extracció de característiques i preparació de dades. El següent pas consisteix en entrenar un model de classificació, com ara una màquina de vectors de suport (SVM), utilitzant les característiques HOG preparades.


In [1]:
import cv2
import glob
import numpy as np
from skimage.feature import hog
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

# Definir les carpetes de fruites fresques i podrides
freshfruits = {
    "freshapples": "fresh",
    "freshbanana": "fresh",
    "freshoranges": "fresh"
}

rottenfruits = {
    "rottenapples": "rotten",
    "rottenbanana": "rotten",
    "rottenoranges": "rotten"
}

# Llistes per emmagatzemar les característiques i etiquetes
data_train = []
labels_train = []

data_test = []
labels_test = []

# Funció per processar les imatges
def process_images(folder_dict, dataset_type='train'):
    for folder, freshness_label in folder_dict.items():
        folder_path = f".gitignore/dataset/{dataset_type}/{folder}/*.png"
        for image_path in glob.glob(folder_path):
            img = cv2.imread(image_path)
            if img is None or img.shape[2] != 3:
                print(f"Imatge no vàlida o amb canals incorrectes: {image_path}")
                continue  # Saltar imatges no vàlides o amb canals incorrectes
            try:
                # Redimensionar la imatge
                resized_img = cv2.resize(img, (64, 32))
                # Convertir a tons de gris
                gray_img = cv2.cvtColor(resized_img, cv2.COLOR_BGR2GRAY)
                # Extracció de característiques HOG
                fd = hog(
                    gray_img,
                    pixels_per_cell=(8, 8),
                    cells_per_block=(2, 2),
                    visualize=False,
                    feature_vector=True,
                    block_norm='L2-Hys'
                )
            except Exception as e:
                print(f"Error processant {image_path}: {e}")
                continue  # Saltar imatges que causin errors en HOG

            # Assignar les dades i etiquetes segons el tipus de dataset
            if dataset_type == 'train':
                data_train.append(fd)
                labels_train.append(freshness_label)
            elif dataset_type == 'test':
                data_test.append(fd)
                labels_test.append(freshness_label)

# Processar les fruites fresques i podrides per al conjunt d'entrenament
process_images(freshfruits, dataset_type='train')
process_images(rottenfruits, dataset_type='train')

# Processar les fruites fresques i podrides per al conjunt de prova
process_images(freshfruits, dataset_type='test')
process_images(rottenfruits, dataset_type='test')

print("----- Conjunt d'Entrenament -----")
print("Total d'imatges processades (train):", len(data_train))
print("Total d'etiquetes (train):", len(labels_train))

print("\n----- Conjunt de Prova -----")
print("Total d'imatges processades (test):", len(data_test))
print("Total d'etiquetes (test):", len(labels_test))

# Convertir les dades i etiquetes a arrays de NumPy
X_train = np.array(data_train)
y_train = np.array(labels_train)

X_test = np.array(data_test)
y_test = np.array(labels_test)

# Codificació de les etiquetes
encoder = LabelEncoder()
y_train_encoded = encoder.fit_transform(y_train)
y_test_encoded = encoder.transform(y_test)

----- Conjunt d'Entrenament -----
Total d'imatges processades (train): 10901
Total d'etiquetes (train): 10901

----- Conjunt de Prova -----
Total d'imatges processades (test): 2698
Total d'etiquetes (test): 2698


In [2]:
from sklearn.svm import SVC
from sklearn.multiclass import OneVsOneClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

# Definir el model One-vs-One amb SVM de kernel lineal
ovo_classifier = OneVsOneClassifier(SVC(kernel='linear', probability=True, random_state=42))

# Entrenar el model
ovo_classifier.fit(X_train, y_train_encoded)

# Predir sobre el conjunt de prova
y_pred_ovo = ovo_classifier.predict(X_test)

# Avaluar el model
accuracy_ovo = accuracy_score(y_test_encoded, y_pred_ovo)
print("----- One-vs-One (Kernel Lineal) -----")
print(f"Accuracy: {accuracy_ovo:.4f}\n")
print("Informe de classificació:")
print(classification_report(y_test_encoded, y_pred_ovo, target_names=encoder.classes_))
print("Matriu de Confusió:")
print(confusion_matrix(y_test_encoded, y_pred_ovo))

----- One-vs-One (Kernel Lineal) -----
Accuracy: 0.7991

Informe de classificació:
              precision    recall  f1-score   support

       fresh       0.79      0.73      0.76      1164
      rotten       0.81      0.85      0.83      1534

    accuracy                           0.80      2698
   macro avg       0.80      0.79      0.79      2698
weighted avg       0.80      0.80      0.80      2698

Matriu de Confusió:
[[ 852  312]
 [ 230 1304]]


In [3]:
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

# Definir el model One-vs-Rest amb SVM de kernel lineal
ovr_classifier = OneVsRestClassifier(SVC(kernel='linear', probability=True, random_state=42))

# Entrenar el model
ovr_classifier.fit(X_train, y_train_encoded)

# Predir sobre el conjunt de prova
y_pred_ovr = ovr_classifier.predict(X_test)

# Avaluar el model
accuracy_ovr = accuracy_score(y_test_encoded, y_pred_ovr)
print("----- One-vs-Rest (Kernel Lineal) -----")
print(f"Accuracy: {accuracy_ovr:.4f}\n")
print("Informe de classificació:")
print(classification_report(y_test_encoded, y_pred_ovr, target_names=encoder.classes_))
print("Matriu de Confusió:")
print(confusion_matrix(y_test_encoded, y_pred_ovr))

----- One-vs-Rest (Kernel Lineal) -----
Accuracy: 0.7991

Informe de classificació:
              precision    recall  f1-score   support

       fresh       0.79      0.73      0.76      1164
      rotten       0.81      0.85      0.83      1534

    accuracy                           0.80      2698
   macro avg       0.80      0.79      0.79      2698
weighted avg       0.80      0.80      0.80      2698

Matriu de Confusió:
[[ 852  312]
 [ 230 1304]]


In [4]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

# Definir el model SVM amb kernel RBF
rbf_classifier = SVC(kernel='rbf', gamma='scale', probability=True, random_state=42)

# Entrenar el model
rbf_classifier.fit(X_train, y_train_encoded)

# Predir sobre el conjunt de prova
y_pred_rbf = rbf_classifier.predict(X_test)

# Avaluar el model
accuracy_rbf = accuracy_score(y_test_encoded, y_pred_rbf)
print("----- SVM amb Kernel RBF -----")
print(f"Accuracy: {accuracy_rbf:.4f}\n")
print("Informe de classificació:")
print(classification_report(y_test_encoded, y_pred_rbf, target_names=encoder.classes_))
print("Matriu de Confusió:")
print(confusion_matrix(y_test_encoded, y_pred_rbf))

----- SVM amb Kernel RBF -----
Accuracy: 0.8758

Informe de classificació:
              precision    recall  f1-score   support

       fresh       0.87      0.84      0.85      1164
      rotten       0.88      0.91      0.89      1534

    accuracy                           0.88      2698
   macro avg       0.88      0.87      0.87      2698
weighted avg       0.88      0.88      0.88      2698

Matriu de Confusió:
[[ 974  190]
 [ 145 1389]]


In [5]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

# Definir el model SVM amb kernel polinòmic
poly_classifier = SVC(kernel='poly', degree=3, coef0=1, probability=True, random_state=42)

# Entrenar el model
poly_classifier.fit(X_train, y_train_encoded)

# Predir sobre el conjunt de prova
y_pred_poly = poly_classifier.predict(X_test)

# Avaluar el model
accuracy_poly = accuracy_score(y_test_encoded, y_pred_poly)
print("----- SVM amb Kernel Polinòmic -----")
print(f"Accuracy: {accuracy_poly:.4f}\n")
print("Informe de classificació:")
print(classification_report(y_test_encoded, y_pred_poly, target_names=encoder.classes_))
print("Matriu de Confusió:")
print(confusion_matrix(y_test_encoded, y_pred_poly))

----- SVM amb Kernel Polinòmic -----
Accuracy: 0.9044

Informe de classificació:
              precision    recall  f1-score   support

       fresh       0.90      0.88      0.89      1164
      rotten       0.91      0.93      0.92      1534

    accuracy                           0.90      2698
   macro avg       0.90      0.90      0.90      2698
weighted avg       0.90      0.90      0.90      2698

Matriu de Confusió:
[[1019  145]
 [ 113 1421]]


# Resum i Conclusions dels Resultats dels Models de Classificació

En aquest projecte, s'han entrenat diversos models de Support Vector Machines (SVM) amb diferents configuracions per classificar imatges de fruites com a **fresques** o **podrides**. A continuació, es presenten les observacions i conclusions derivades dels resultats obtinguts.

**1. Similitud entre One-vs-One i One-vs-Rest**

Els models **One-vs-One (OvO)** i **One-vs-Rest (OvR)** amb un **kernel lineal** han produït resultats idèntics. Aquesta coincidència és esperada en un context de classificació binària, on només hi ha dues classes a considerar. En aquest cas, tant OvO com OvR implementen essencialment la mateixa estratègia de discriminació, ja que només necessiten distingir entre dues categories. Per tant, no hi ha diferència significativa en el rendiment dels dos enfocaments quan es treballa amb dues classes.

**2. Impacte del Kernel en el Rendiment dels Models**

**a. Kernel Lineal vs. Kernels No Lineals**

Els models amb un **kernel lineal** (tant OvO com OvR) han demostrat un rendiment adequat però amb marge de millora. En contrast, l'ús de **kernels no lineals** com el **RBF** i el **Polinòmic** ha resultat en una millora substancial de la precisió i altres mètriques de rendiment.

- **Kernel RBF:** Aquest kernel ha proporcionat una millora significativa en l'exactitud i en les mètriques de **precision**, **recall** i **f1-score**. Això indica que el model és capaç de capturar relacions més complexes i no lineals entre les característiques de les imatges, millorant la seva capacitat de discriminació entre les dues classes.

- **Kernel Polinòmic:** Amb la millor rendibilitat entre els models entrenats, el kernel polinòmic ha demostrat la seva eficàcia per modelar interaccions més complexes dins de les dades. Aquest tipus de kernel permet al model adaptar-se millor a les particularitats dels patrons presentats en les imatges, resultant en una classificació més precisa i amb menys errors.

**b. Reducció d'Errors amb Kernels No Lineals**

Els models amb **kernels no lineals** han reduït notablement els tipus d'errors com ara classificar erròniament les fruites **fresques** com **podrides** i viceversa. Aquesta millora és essencial per aplicacions pràctiques on la precisió en la classificació té un impacte directe en la qualitat dels productes i en l'eficiència dels processos posteriors.

**3. Conclusió Final**

1. **Eficiència de les Estratègies Multiclasse en Classificació Binària:**
   - En classificacions binàries, les estratègies **One-vs-One** i **One-vs-Rest** són equivalentes. Això simplifica la selecció de l'estratègia multiclasse en projectes amb dues classes, ja que no cal invertir temps en comparar els diferents enfocaments en aquests casos.

2. **Selecció del Kernel Apropiat:**
   - L'ús de **kernels no lineals**, especialment el **Polinòmic**, ha demostrat ser molt beneficiós per millorar el rendiment del model. Per tant, en problemes similars on les dades no són linealment separables, és recomanable optar per kernels que puguin capturar relacions més complexes.

3. **Millora Continua:**
   - Tot i els bons resultats obtinguts amb els kernels no lineals, es pot explorar l'ajustament de hiperparàmetres mitjançant tècniques com **GridSearchCV** per optimitzar encara més el rendiment dels models.
   - Considerar la implementació de **validació creuada** pot proporcionar una estimació més robusta del rendiment del model i assegurar que els resultats són generalitzables a dades noves.

4. **Implementacions Pràctiques:**
   - Els models amb kernels no lineals, especialment el polinòmic, són els més adients per a aplicacions que requereixen alta precisió en la classificació, com ara el control de qualitat en la indústria alimentària.

En resum, la selecció adequada de l'estratègia multiclasse i del tipus de kernel té un impacte significatiu en el rendiment dels models de classificació. En casos de classificació binària, com el present, escollir un kernel no lineal com el polinòmic pot millorar substancialment la precisió del model, assegurant una millor classificació de les imatges de fruites fresques i podrides.
